In [ ]:
# Import Modules
import pandas as pd
import plotly.express as px
import swifter
from tqdm import tqdm

from src.data_functions import load_sightings_data, add_features_sightings_data


def plot_confusion_matrix_plotly(y_true, y_pred, labels=None, normalize=False):
    cm = confusion_matrix(y_true, y_pred, labels=labels)

    if normalize:
        cm = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis]
        cm = np.round(cm, 2)

    if labels is None:
        labels = sorted(list(set(y_true) | set(y_pred)))

    cm_df = pd.DataFrame(cm, index=labels, columns=labels)

    fig = px.imshow(
        cm_df,
        text_auto=True,
        color_continuous_scale="Reds",
        aspect="auto",
        labels=dict(x="Predicted", y="Actual", color="Count"),
    )

    fig.update_xaxes(side="top")
    fig.update_layout(title="Confusion Matrix", font=dict(size=14))

    return fig, cm


def plot_classification_report(classification_metrics):

    fig = px.line(title="Classification Metrics at Different Thresholds")
    fig.add_scatter(
        x=classification_metrics["THRESHOLD"],
        y=classification_metrics["ACCURACY"],
        name="Accuracy",
        marker_color="#ff006e",
    )
    fig.add_scatter(
        x=classification_metrics["THRESHOLD"],
        y=classification_metrics["PRECISION"],
        name="Precision",
        marker_color="#8338ec",
    )
    fig.add_scatter(
        x=classification_metrics["THRESHOLD"],
        y=classification_metrics["RECALL"],
        name="Recall",
        marker_color="#3a86ff",
    )
    fig.add_scatter(
        x=classification_metrics["THRESHOLD"],
        y=classification_metrics["F1"],
        name="F1 Score",
        marker_color="#ffbe0b",
    )
    return fig


def threshold_metrics_plot(y_true, y_proba, steps=100):
    thresholds = np.linspace(0, 1, steps)
    precisions = []
    recalls = []
    f1s = []
    accuracies = []

    for t in tqdm(thresholds):
        y_pred = (y_proba >= t).astype(int)
        precisions.append(precision_score(y_true, y_pred, zero_division=0))
        recalls.append(recall_score(y_true, y_pred))
        f1s.append(f1_score(y_true, y_pred))
        accuracies.append(accuracy_score(y_true, y_pred))

    threshold_lookup = pd.DataFrame(
        {
            "THRESHOLD": thresholds,
            "ACCURACY": accuracies,
            "PRECISION": precisions,
            "RECALL": recalls,
            "F1": f1s,
        }
    )

    return threshold_lookup


def plot_threshold_metrics(df, min_precision=0.4):
    max_possible_precision = df["PRECISION"].max()

    # Adjust min_precision if it's too high
    if min_precision > max_possible_precision:
        print(
            f"⚠️ Requested min_precision={min_precision} "
            f"is higher than max precision={max_possible_precision:.3f}. Lowering requirement."
        )
        min_precision = max_possible_precision

    # Filter rows that meet min precision requirement
    valid_df = df[df["PRECISION"] >= min_precision]

    if not valid_df.empty:
        best_row = valid_df.loc[valid_df["RECALL"].idxmax()]
    else:
        best_row = df.loc[df["RECALL"].idxmax()]  # fallback: max recall

    best_threshold = best_row["THRESHOLD"]

    return best_threshold

***

### Paths + Parameters

In [ ]:
# Parameters
SIGHTINGS_PATH = (
    "../../data/processed/ORCA_SIGHTINGS/ORCA_SIGHTINGS.parquet"  # Data Paths
)
H3_RESOLUTION = 5  # Target Resolution
START_DATE = None  # Optional: set start date for generating absence rows
END_DATE = None  # Optional: set end date
POD_TYPE = "SRKW"

***

### Open Sightings Data and Add Initial Feature Set

1. Map lat/lon → configurable low-res H3.
2. Aggregate to one row per (H3 cell, date).
3. Add report_count, orca_present columns.
4. Fill missing dates for all cells.
5. Add seasonality encoding (sin/cos of DOY, MONTH, WOY).

In [ ]:
# Load Sightings Data
df_model = load_sightings_data(
    SIGHTINGS_PATH, POD_TYPE, H3_RESOLUTION, START_DATE, END_DATE
)

# Preprocess Data
df_model = add_features_sightings_data(df_model)

# display(df_model.head())
# display(df_model["presence"].value_counts())

*** 

## Fit Simple Model - H3 Grid & DOY as Predictions

This is to check if there is predictive power in the H3 Grid + Date Combo Alone -> there should be since SRKW whales return to similar locations year-over-year.

We will test Logistic Regression and XGBoost

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import sparse
from sklearn.ensemble import IsolationForest
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    roc_auc_score,
    precision_score,
    recall_score,
    f1_score,
    accuracy_score,
)
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from xgboost import XGBClassifier

In [ ]:
# We Will Split Using Date - Sinc ethis is a Time Series and we may use different kinds of features later
split_date = "2025-01-01"  # example date for split

# 1. Train/Test Split
train_idx, test_idx = df_model["DATE"] < split_date, df_model["DATE"] >= split_date

# 2. Split Data
train, test = df_model[train_idx], df_model[test_idx]

# 3. Define Temporal Columns
temporal_cols = ["DOY", "DOY_SIN", "DOY_COS"]

# 4. Define Categorical Columns for One-Hot Encoding
categorical_cols = ["H3_CELL"]

In [ ]:
# 1. Fit One-Hot Encoder to Categorical Columns
ohe = OneHotEncoder(sparse_output=True, handle_unknown="ignore")
h3_sparse = ohe.fit_transform(df_model[categorical_cols])

# 2. Standardize Continuous Columns
scaler = StandardScaler()
X_temporal = scaler.fit_transform(df_model[temporal_cols])

# 3. Stack sparse H3 one-hot with dense temporal features
X_sparse = sparse.hstack([h3_sparse, X_temporal])

# 4. Convert to CSR format (fast row slicing)
X_sparse = X_sparse.tocsr()

# 5a. Split Train
X_train = X_sparse[train_idx.values]
y_train = df_model["presence"].loc[train_idx]

# 5b. Split Test
X_test = X_sparse[test_idx.values]
y_test = df_model["presence"].loc[test_idx]

In [ ]:
###########################################

# 1. Fit Models
## 1a. Logistic Regression
lr_model = LogisticRegression(max_iter=1000, class_weight="balanced", n_jobs=-1)
lr_model.fit(X_train, y_train)

###########################################

In [ ]:
## 1b. XGBoost
xg_model = XGBClassifier(
    n_estimators=500,  # number of trees
    learning_rate=0.05,  # step size shrinkage
    max_depth=6,  # tree depth
    subsample=0.8,  # row sampling
    colsample_bytree=0.8,  # feature sampling
    scale_pos_weight=(len(y_train) - sum(y_train))
    / sum(y_train),  # handle imbalance like class_weight
    eval_metric="logloss",  # metric for eval sets
    n_jobs=-1,
    random_state=42,
)
xg_model.fit(X_train, y_train)

###########################################

In [ ]:
###########################################

# 2a. Evaluate Model - Logistic Regression
lr_y_pred = lr_model.predict(X_test)
lr_y_proba = lr_model.predict_proba(X_test)[:, 1]

# Get Classification Metrics
lr_classification_metrics = threshold_metrics_plot(y_test, lr_y_proba, steps=100)

# Get Best Threshold Using Recall Balanced by Precision
lr_best_threshold = plot_threshold_metrics(lr_classification_metrics, min_precision=0.5)

###########################################

# 2b. Evaluate Model - XGBoost
xg_y_pred = xg_model.predict(X_test)
xg_y_proba = xg_model.predict_proba(X_test)[:, 1]

# Get Classification Metrics
xg_classification_metrics = threshold_metrics_plot(y_test, xg_y_proba, steps=100)

# Get Best Threshold Using Recall Balanced by Precision
xg_best_threshold = plot_threshold_metrics(xg_classification_metrics, min_precision=0.5)

###########################################

In [ ]:
# fig = plot_classification_report(lr_classification_metrics)
# fig.add_scatter(x = lr_best_threshold)
# fig.show()

# lr_y_pred_w_thresh = np.where(lr_y_proba > lr_best_threshold, 1, 0)
# fig, cm = plot_confusion_matrix_plotly(
#     y_test, lr_y_pred_w_thresh, labels=None, normalize=False
# )
# fig.show()

In [ ]:
# xg_y_pred_w_thresh = np.where(xg_y_proba > xg_best_threshold, 1, 0)
# fig, cm = plot_confusion_matrix_plotly(
#     y_test, xg_y_pred_w_thresh, labels=None, normalize=False
# )
# fig.show()

In [ ]:
# cm

*** 

## What About Isolation Forest? 

Right now positives are ~0.6% of the dataset.
In that regime, even a “balanced” classifier spends most of its energy learning the negative class and basically ignores the positives unless you force it to care.
Anomaly detection flips that around — instead of “learn both classes equally,” you learn what “normal” looks like (negatives), then flag things that look different.
This is exactly what fraud detection, rare disease screening, and whale detection in sparse conditions do.

In [ ]:
# Train only on negative class
# Ensure boolean mask alignment
mask = (y_train == 0).values  # <-- converts to NumPy array
X_train_neg = X_train[mask]

# Or if X_train is a NumPy array already:
X_train_neg = X_train[mask, :]

iso = IsolationForest(
    n_estimators=200, contamination=0.006, random_state=42  # approximate positive rate
)
iso.fit(X_train_neg)

# Anomaly score (higher = more anomalous)
scores = -iso.score_samples(X_test)

In [ ]:
# Pick threshold to match desired recall/precision
threshold = np.percentile(scores, 99.4)  # tweak based on desired recall
y_pred_anom = (scores >= threshold).astype(int)

In [ ]:
cm = confusion_matrix(y_test, y_pred_anom)
print(cm)

Still not great, our best bet is to do feature engineering...

***

## Feature Engineering
Even simple models benefit from a few extra features. Ideas to start with:
- Temporal features: DOY, WOY, MONTH, YEAR (already there), maybe sine/cosine transforms for seasonal cycles.
- Lagged presence features: e.g., presence in the same H3 cell 1, 2, 3 days ago. Useful for persistence.
- Rolling/aggregated features: 3-day or 7-day rolling sum or mean of sightings per H3 cell.
- Neighbor info: later, you can include aggregated presence in neighboring H3 cells to capture spatial autocorrelation.

In [ ]:
# Make sure data is sorted by H3_CELL and DATE
df_model = df_model.sort_values(["H3_CELL", "DATE"]).reset_index(drop=True)

# Lags: presence 1,2,3, 26, 52, 56 days ago
lag_days = [1, 2, 3, 7, 13, 26, 52, 56]
for lag in lag_days:
    df_model[f"lag_{lag}"] = df_model.groupby("H3_CELL")["presence"].shift(lag)

# Rolling sums: last 3 and 7 days
rolling_windows = [3, 7, 14]
for window in rolling_windows:
    df_model[f"roll_{window}"] = (
        df_model.groupby("H3_CELL")["presence"]
        .shift(1)
        .rolling(window=window, min_periods=1)
        .sum()
    )

# Fill NaNs with 0 (first few days have no lag/rolling)
lag_roll_cols = [f"lag_{l}" for l in lag_days] + [f"roll_{w}" for w in rolling_windows]
df_model[lag_roll_cols] = df_model[lag_roll_cols].fillna(0)

In [ ]:
df_model.columns

In [ ]:
# Identify Categorical Columns
categorical_cols = ["H3_CELL", "MONTH"]

# Identify Temporal Columns
temporal_cols = [
    "DOY",
    "DOW",
    "WOY",
    "MONTH",
    "YEAR",
    "MONTH_SIN",
    "MONTH_COS",
    "DOY_SIN",
    "DOY_COS",
    "WOY_SIN",
    "WOY_COS",
    "lag_1",
    "lag_2",
    "lag_3",
    "lag_7",
    "lag_13",
    "lag_26",
    "lag_52",
    "lag_56",
    "roll_3",
    "roll_7",
    "roll_14",
]

In [ ]:
# Sparse one-hot for H3_CELL
ohe = OneHotEncoder(sparse_output=True, handle_unknown="ignore")
h3_sparse = ohe.fit_transform(df_model[categorical_cols])

# Scale temporal features
# temporal_cols = ["DOY", "WOY", "MONTH", "YEAR"]
temporal_cols = ["DOY", "WOY", "YEAR", "MONTH_SIN", "MONTH_COS", "DOY_SIN", "DOY_COS"]
scaler = StandardScaler()
X_temporal = scaler.fit_transform(df_model[temporal_cols])

# Combine sparse + temporal + lag/rolling
X_dense = df_model[lag_roll_cols].values  # dense numeric
X_sparse_full = sparse.hstack([h3_sparse, X_temporal, X_dense])

In [ ]:
train_idx = df_model["DATE"] < split_date
test_idx = df_model["DATE"] >= split_date

# Convert to CSR format (fast row slicing)
X_sparse_full = X_sparse_full.tocsr()

X_train = X_sparse_full[train_idx.values]
y_train = df_model["presence"].loc[train_idx]

X_test = X_sparse_full[test_idx.values]
y_test = df_model["presence"].loc[test_idx]

In [ ]:
## 1b. XGBoost
xg_model = XGBClassifier(
    n_estimators=500,  # number of trees
    learning_rate=0.05,  # step size shrinkage
    max_depth=6,  # tree depth
    subsample=0.8,  # row sampling
    colsample_bytree=0.8,  # feature sampling
    scale_pos_weight=(len(y_train) - sum(y_train))
    / sum(y_train),  # handle imbalance like class_weight
    eval_metric="logloss",  # metric for eval sets
    n_jobs=-1,
    random_state=42,
)
xg_model.fit(X_train, y_train)

###########################################

In [ ]:
def recursive_forecast_clean(
    model,
    df_last,
    ohe,
    scaler,
    lag_roll_cols,
    temporal_cols,
    h3_col="H3_CELL",
    n_days=7,
    thresh_=0.5,
):

    forecasts = []
    df_forecast = df_last.copy()

    short_lags = [1, 2, 3, 7, 13, 26, 52, 56]
    rolling_windows = [3, 7, 14]

    for day in range(1, n_days + 1):
        # Increment date and update temporal features
        df_forecast["DATE"] = df_forecast["DATE"] + pd.Timedelta(days=1)
        df_forecast["DOY"] = df_forecast["DATE"].dt.dayofyear
        df_forecast["WOY"] = df_forecast["DATE"].dt.isocalendar().week
        df_forecast["MONTH"] = df_forecast["DATE"].dt.month
        df_forecast["YEAR"] = df_forecast["DATE"].dt.year

        df_forecast["MONTH_SIN"] = np.sin(2 * np.pi * df_forecast["MONTH"] / 12)
        df_forecast["MONTH_COS"] = np.cos(2 * np.pi * df_forecast["MONTH"] / 12)

        df_forecast["DOY_SIN"] = np.sin(2 * np.pi * df_forecast["MONTH"] / 12)
        df_forecast["DOY_COS"] = np.cos(2 * np.pi * df_forecast["MONTH"] / 12)

        df_forecast["WOY_SIN"] = np.sin(2 * np.pi * df_forecast["WOY"] / 52)
        df_forecast["WOY_COS"] = np.cos(2 * np.pi * df_forecast["WOY"] / 52)

        # Transform features
        X_temporal = scaler.transform(df_forecast[temporal_cols])
        h3_sparse = ohe.transform(df_forecast[[h3_col, "MONTH"]])
        X_dense = df_forecast[lag_roll_cols].values
        X_input = sparse.hstack([h3_sparse, X_temporal, X_dense])

        # Predict
        proba = model.predict_proba(X_input)[:, 1]
        presence_pred = (proba >= thresh_).astype(float)

        df_forecast["presence_pred"] = presence_pred
        df_forecast["proba"] = proba
        forecasts.append(df_forecast[["DATE", h3_col, "presence_pred", "proba"]].copy())

        # Update lag/rolling for next day
        for lag in short_lags:
            df_forecast[f"lag_{lag}"] = (
                df_forecast.groupby(h3_col)["presence_pred"].shift(lag).fillna(0)
            )
        for window in rolling_windows:
            df_forecast[f"roll_{window}"] = (
                df_forecast.groupby(h3_col)["presence_pred"]
                .shift(1)
                .rolling(window=window, min_periods=1)
                .sum()
                .fillna(0)
            )

    return pd.concat(forecasts).reset_index(drop=True)

In [ ]:
# All H3 cells in training data (or full dataset)
all_h3_cells = df_model["H3_CELL"].unique()
train_max_date = df_model[df_model["DATE"] <= split_date]["DATE"].max()

# Create df_last with one row per H3 cell
df_last = pd.DataFrame({"H3_CELL": all_h3_cells})
df_last["DATE"] = train_max_date

# Add lag/rolling columns initialized to 0
for col in lag_roll_cols:
    df_last[col] = 0

In [ ]:
# Number of days in test set
n_days = df_model[df_model["DATE"] > split_date]["DATE"].nunique()

forecast_df = recursive_forecast_clean(
    model=xg_model,
    df_last=df_last,
    ohe=ohe,
    scaler=scaler,
    lag_roll_cols=lag_roll_cols,
    temporal_cols=temporal_cols,
    n_days=n_days,
    thresh_=0.1,
)

In [ ]:
# Only rows that actually exist in test set
df_eval = pd.merge(
    forecast_df,
    df_model[test_idx][["H3_CELL", "DATE", "presence"]],
    on=["H3_CELL", "DATE"],
    how="inner",  # keep only observed rows
)
# df_eval["presence_pred"] = np.where(df_eval["proba"] > 0.15, 1, 0)

y_true = df_eval["presence"]
y_pred = df_eval["presence_pred"]

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score

print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))

print("\nClassification Report:")
print(classification_report(y_true, y_pred))

print("\nROC-AUC Score:")
print(roc_auc_score(y_true, df_eval["proba"]))

In [ ]:
import numpy as np
from sklearn.metrics import precision_recall_curve, f1_score

# True labels and predicted probabilities
y_true = df_eval["presence"]
y_probs = df_eval["proba"]

thresholds = np.arange(0.01, 1.0, 0.01)
best_thresh = 0.5
best_f1 = 0

for t in thresholds:
    y_pred = (y_probs >= t).astype(int)
    f1 = f1_score(y_true, y_pred)
    if f1 > best_f1:
        best_f1 = f1
        best_thresh = t

print(f"Best threshold for F1: {best_thresh:.2f} (F1={best_f1:.3f})")

In [ ]:
thresholds = np.arange(0.05, 0.5, 0.05)  # lower thresholds since data is imbalanced
results = []

for t in thresholds:
    forecast_df = recursive_forecast_clean(
        model=xg_model,
        df_last=df_last,
        ohe=ohe,
        scaler=scaler,
        lag_roll_cols=lag_roll_cols,
        temporal_cols=temporal_cols,
        n_days=n_days,
        thresh_=t,
    )

    # Merge with observed test data (only rows with true presence)
    df_eval = pd.merge(
        forecast_df,
        df_model[test_idx][["H3_CELL", "DATE", "presence"]],
        on=["H3_CELL", "DATE"],
        how="inner",
    )

    y_true = df_eval["presence"]
    y_pred = df_eval["presence_pred"]

    f1 = f1_score(y_true, y_pred)
    results.append((t, f1))

# Find best threshold
best_thresh, best_f1 = max(results, key=lambda x: x[1])
print(f"Best threshold in recursive forecast: {best_thresh:.2f} (F1={best_f1:.3f})")

In [ ]:
best_thresh

In [ ]:
# Get Forecast w/Best Threshold
forecast_df = recursive_forecast_clean(
    model=xg_model,
    df_last=df_last,
    ohe=ohe,
    scaler=scaler,
    lag_roll_cols=lag_roll_cols,
    temporal_cols=temporal_cols,
    n_days=n_days,
    thresh_=0.1,  # best_thresh,
)

In [ ]:
#######################################################

# Only rows that actually exist in test set
df_eval = pd.merge(
    forecast_df,
    df_model[test_idx][["H3_CELL", "DATE", "presence"]],
    on=["H3_CELL", "DATE"],
    how="inner",  # keep only observed rows
)
df_eval["presence_pred"] = np.where(df_eval["proba"] > 0.05, 1, 0)

y_true = df_eval["presence"]
y_pred = df_eval["presence_pred"]

#######################################################

print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))

print("\nClassification Report:")
print(classification_report(y_true, y_pred))

print("\nROC-AUC Score:")
print(roc_auc_score(y_true, df_eval["proba"]))

#######################################################

In [ ]:
from src.data_functions import h3_to_polygon
import geopandas as gpd

In [ ]:
forecast_h3 = forecast_df.copy()  # [forecast_df.DATE == "2025-07-24"]
forecast_h3["geometry"] = forecast_h3["H3_CELL"].apply(h3_to_polygon)
forecast_h3 = gpd.GeoDataFrame(forecast_h3, geometry="geometry", crs="EPSG:4326")

forecast_h3["proba_scaled"] = (forecast_h3["proba"] - forecast_h3["proba"].min()) / (
    forecast_h3["proba"].max() - forecast_h3["proba"].min()
)
forecast_h3["WOY"] = forecast_h3["DATE"].dt.isocalendar().week
forecast_h3["YEAR"] = forecast_h3["DATE"].dt.year

In [ ]:
forecast_h3[forecast_h3.DATE == "2025-07-24"].explore(
    "presence_pred", cmap="cool"
).save("test.html")

In [ ]:
# year_ = 2025

# sightings_filt = sightings[(sightings.POD_TYPE == pod_type) & (sightings.YEAR == year_)]

# sightings_filt.WOY.unique()

In [ ]:
# m = forecast_h3_date.explore(
#     "proba_scaled",
#     cmap="cool",
#     tiles="CartoDB positron",
# )
# m = sightings_filt.explore(m=m)
# # m

In [ ]:
# Gaussian Smooth Forecasts
import h3
import numpy as np
import pandas as pd

# df_forecast has columns: H3_CELL, proba
cells = forecast_h3["H3_CELL"].unique()
cell_coords = np.array([h3.cell_to_latlng(cell) for cell in cells])  # lat/lon

In [ ]:
from scipy.spatial.distance import cdist

# Choose bandwidth in km (sigma)
sigma_km = 5.0


# Compute pairwise distances (Haversine) in km
def haversine(latlon1, latlon2):
    lat1, lon1 = np.radians(latlon1[:, 0]), np.radians(latlon1[:, 1])
    lat2, lon2 = np.radians(latlon2[:, 0]), np.radians(latlon2[:, 1])
    dlat = lat2[:, None] - lat1[None, :]
    dlon = lon2[:, None] - lon1[None, :]
    a = (
        np.sin(dlat / 2) ** 2
        + np.cos(lat1[None, :]) * np.cos(lat2[:, None]) * np.sin(dlon / 2) ** 2
    )
    c = 2 * np.arcsin(np.sqrt(a))
    return 6371 * c  # Earth radius in km


dist_matrix = haversine(cell_coords, cell_coords)  # shape: (n_cells, n_cells)

# Gaussian weights
weights = np.exp(-(dist_matrix**2) / (2 * sigma_km**2))

In [ ]:
# Original probabilities in the same order as `cells`
proba_vector = forecast_h3.groupby("H3_CELL")["proba"].mean().reindex(cells).values

# Smoothed probability
smoothed_proba = weights @ proba_vector / weights.sum(axis=1)

# Add back to dataframe
forecast_h3["proba_smooth"] = forecast_h3["H3_CELL"].map(
    dict(zip(cells, smoothed_proba))
)

forecast_h3["proba_smooth_scale"] = (
    forecast_h3["proba_smooth"] - forecast_h3["proba_smooth"].min()
) / (forecast_h3["proba_smooth"].max() - forecast_h3["proba_smooth"].min())

In [ ]:
# Only rows that actually exist in test set
df_eval = pd.merge(
    forecast_h3,
    df_model[test_idx][["H3_CELL", "DATE", "presence"]],
    on=["H3_CELL", "DATE"],
    how="inner",  # keep only observed rows
)
df_eval["presence_smooth"] = np.where(df_eval["proba_smooth"] > 0.05, 1, 0)

y_true = df_eval["presence"]
y_pred = df_eval["presence_smooth"]

from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score

print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))

print("\nClassification Report:")
print(classification_report(y_true, y_pred))

print("\nROC-AUC Score:")
print(roc_auc_score(y_true, df_eval["proba"]))

In [ ]:
forecast_h3[forecast_h3.DATE == "2025-07-24"].explore(
    "proba_smooth_scale", cmap="cool"
).save("test.html")

In [ ]:
# 1️⃣ Include neighboring cells
# Add features like presence in adjacent H3 cells or rolling sums over parent grids.
# This helps the model predict a sighting in a nearby cell even if that cell itself hasn’t seen a sighting yet.
# 2️⃣ Include environmental / spatial covariates
# Latitude/longitude (or encoded features)
# Distance to known hotspots
# Prey availability, tides, or other relevant covariates
# This gives the model some signal for predicting outside the “usual” cells.
# 3️⃣ Adjust class imbalance / threshold
# You could slightly lower the threshold for all other H3 cells to catch rare presences.
# Or use a probability calibration / class-weighting method to avoid underestimating rare events.
# 4️⃣ Consider spatial smoothing / Gaussian kernel
# Post-process predicted probabilities by smoothing across neighbors, so a hotspot “spills” into adjacent cells.
# This is a neat trick if you want to maintain simplicity but improve spatial coverage.